<a href="https://colab.research.google.com/github/HwangHanJae/recommender_system/blob/main/book_Recommender_Systems/Model_Base_SVD_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 특이값 분해(Singular Value Decomposition) - SVD

참고링크1 : https://data-science-hi.tistory.com/82  
참고링크2 : https://velog.io/@dlskawns/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C-%EC%9D%B4%EB%A1%A0-%EC%BB%A8%ED%85%90%EC%B8%A0-%EA%B8%B0%EB%B0%98%EC%B6%94%EC%B2%9CCB-%ED%98%91%EC%97%85%ED%95%84%ED%84%B0%EB%A7%81CF  


특이값 분해는 ($m \times n$)으로 이루어진 행렬($R$)에 대해 3개의 행렬로 분해하는 것을 말합니다.

$$R = $Q∑P^{T}$

이때 $Q$는 ($m \times k$), $∑$는 ($k \times k$), $P$는 ($n \times k$)의 행렬이 됩니다.

행렬 $Q$는 $RR^{T}$의 $k$개의 가장 큰 고유벡터를 포함하고  
행렬 $P$는 $R^{T}R$의 $k$개의 가장 큰 고유벡터를 포함합니다.  
그리고 행렬 $∑$는 각 행렬의 $k$개의 가장 큰 고윳값의 제곱근을 포함합니다.  
- 제곱근은 음이 아닙니다.

$Q$, $P$는 차원축소가 되면서 각각의 $k$차원의 잠재요인을 담고 있습니다.

3개로 행렬분해된 행렬을 2개로 정의하여 사용할 수 있습니다.

$$U = Q∑$$
$$V = P$$

그리고 $\hat{R}$을 예측한다면 이렇게 정의할 수 있습니다.

$$\hat{R} ≈ UV^{T}$$

행렬분해를 이용하면 누락된 값(NULL)을 채워 사용자가 아직 평가하지 않은 아이템의 평점을 예측하여 추천에 활용할 수 있습니다.

위처럼 SVD를 활용하여 차원을 축소하고 다시 행렬을 완성시킵니다. 
그리고 누락된 값에 대해서 완성된 행렬에서 채워주고 그 행렬을 다시 SVD를 이용하여 값이 최적화 될 때까지 반복하면서 평점을 채울 수 있습니다.

그리고 bias를 줄이기 위하여 초기에 SVD를 수행할 때 누락된 값을 0으로 채우지 않고 행의 평균으로 채워 bias를 줄일 수 있습니다.

과정을 정리하면 다음과 같습니다.

1. 초기화 : $R$의 $i$번째 행에서 누락된 항목을 초기화 해 해당 행의 평균 $μ_{i}$가 되도록 $R_{f}$를 만듭니다.
2. 반복 1 : $Q∑P^{T}$ 형식으로 $R_{f}$의 rank-$k$ SVD를 수행합니다.
3. 반복 2 : 누락된 $R_{f}$ 항목만 $QΣP^{T}$의 해당 값으로 재조정하고 최적화가 될 때까지 반복 1로 이동합니다.

In [36]:
import numpy as np
import pandas as pd

R = pd.DataFrame([[1, -1, 1, -1, 1, -1],
     [1, 1, None, -1, -1, -1],
     [None, 1, 1, -1, -1, None],
     [-1, -1, -1, 1, 1, 1],
     [-1, None, -1, 1, 1, 1]])
#각 행의 평균
m = R.mean(axis=1)
#원본 행렬 R에 평균 채우기
R_f = R.T.fillna(m).T

In [39]:
null_index = []
for i in R.index:
  for j in R.columns:
    if R.isnull().loc[i,j]:
      null_index.append((i, j))
print(null_index)

[(1, 2), (2, 0), (2, 5), (4, 1)]


In [29]:
#sklearn의 TruncatedSVD를 사용
from sklearn.decomposition import TruncatedSVD

SVD = TruncatedSVD(n_components=2)
U = SVD.fit_transform(R_f)
Sigma = SVD.explained_variance_ratio_
V = SVD.components_

In [31]:
U.dot(V)

array([[ 1.05920375, -1.16043927,  0.97162425, -0.85150512,  0.80400167,
        -1.05920375],
       [ 0.66357584,  0.90392646,  0.58812017, -0.92420035, -1.12439891,
        -0.66357584],
       [ 0.43000805,  0.96228872,  0.37635832, -0.68910774, -1.104505  ,
        -0.43000805],
       [-0.94246874, -0.81805799, -0.84117983,  1.20103733,  1.13200073,
         0.94246874],
       [-1.02900718, -0.20950227, -0.9270483 ,  1.14752076,  0.55345809,
         1.02900718]])

In [41]:
epochs = 5
SVD = TruncatedSVD(n_components=2)
for epoch in range(epochs):
  U = SVD.fit_transform(R_f)
  V = SVD.components_

  SVD_R = pd.DataFrame(U.dot(V))
  for i, j in null_index:
    R_f.loc[i, j] = SVD_R.loc[i, j]
  display(R_f)

,0,1,2,3,4,5
0,1.000000,-1.000000,1.00000,-1.0,1.0,-1.000000
1,1.000000,1.000000,0.58812,-1.0,-1.0,-1.000000
2,0.430008,1.000000,1.00000,-1.0,-1.0,-0.430008
3,-1.000000,-1.000000,-1.00000,1.0,1.0,1.000000
4,-1.000000,-0.209502,-1.00000,1.0,1.0,1.000000


,0,1,2,3,4,5
0,1.000000,-1.000000,1.000000,-1.0,1.0,-1.000000
1,1.000000,1.000000,0.882606,-1.0,-1.0,-1.000000
2,0.667665,1.000000,1.000000,-1.0,-1.0,-0.667665
3,-1.000000,-1.000000,-1.000000,1.0,1.0,1.000000
4,-1.000000,-0.509155,-1.000000,1.0,1.0,1.000000


,0,1,2,3,4,5
0,1.00000,-1.000000,1.00000,-1.0,1.0,-1.00000
1,1.00000,1.000000,0.98729,-1.0,-1.0,-1.00000
2,0.80085,1.000000,1.00000,-1.0,-1.0,-0.80085
3,-1.00000,-1.000000,-1.00000,1.0,1.0,1.00000
4,-1.00000,-0.699515,-1.00000,1.0,1.0,1.00000


,0,1,2,3,4,5
0,1.000000,-1.000000,1.000000,-1.0,1.0,-1.000000
1,1.000000,1.000000,1.018798,-1.0,-1.0,-1.000000
2,0.877234,1.000000,1.000000,-1.0,-1.0,-0.877234
3,-1.000000,-1.000000,-1.000000,1.0,1.0,1.000000
4,-1.000000,-0.815587,-1.000000,1.0,1.0,1.000000


,0,1,2,3,4,5
0,1.00000,-1.000000,1.000000,-1.0,1.0,-1.00000
1,1.00000,1.000000,1.023616,-1.0,-1.0,-1.00000
2,0.92256,1.000000,1.000000,-1.0,-1.0,-0.92256
3,-1.00000,-1.000000,-1.000000,1.0,1.0,1.00000
4,-1.00000,-0.886212,-1.000000,1.0,1.0,1.00000


5~10정도의 반복으로 누락된 값을 채울 수 있습니다.

MSE, RMSE, MAE와 같은 평가지표를 활용하면 해당 모델의 성능을 측정할 수 있고, 여기서는 초기화 과정에서 행의 평균을 이용하였지만, 행렬의 많은 부분이 누락되었을 경우에는 이웃 모델과 같은 방법을 사용하여 초기화 과정에 이용하면 수렴속도가 빨라지고 더욱 정확한 결과를 얻을 수 있습니다.